In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

https://www.99acres.com/api-aggregator/srp/search?property_type=1&area_unit=1&platform=DESKTOP&moduleName=GRAILS_SRP&workflow=GRAILS_SRP&page_size=25&page=2&city=25&preference=S&res_com=R&seoUrlType=DEFAULT

https://www.99acres.com/api-aggregator/psuedoData?prop_id=E52696704&stage=load

In [32]:
count = 1
ID_list = []
while(count >= 1):
  url = f"https://www.99acres.com/api-aggregator/srp/search?property_type=1&area_unit=1&platform=DESKTOP&moduleName=GRAILS_SRP&workflow=GRAILS_SRP&page_size=30&page={count}&city=25&preference=S&res_com=R&seoUrlType=DEFAULT"
  response = requests.get(url)
  if response.json()['status']['code'] != 1:
      break
  else:
    ids = response.json()['properties']
    c = 1
    for id in ids:
      if c != 31:
        ID_list.append(id['PROP_ID'])
        c += 1
  count += 1

print(len(ID_list))

16800


In [33]:
url = "https://www.99acres.com/api-aggregator/psuedoData?prop_id=K51945048&stage=load"

response = requests.get(url)